In [ ]:
import tensorflow as tf
from keras.models import load_model
from keras.saving.legacy.hdf5_format import load_model
import pickle
import pandas as pd
import numpy as np

### Loading the models

In [16]:
## Load the trained model, scaler pickle, onehot_encoder
model = load_model('model.h5')

## Load the encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
        label_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
        label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
        scaler = pickle.load(file)

AttributeError: module 'keras.src.backend' has no attribute 'floatx'

### Random input from data or yr own values

In [7]:
## Example input data

input_data = {
        'CreditScore': 600,
        'Geography' : 'France', 
        'Gender' : 'Male',
        'Age': 40, 
        'Tenure': 3, 
        'Balance': 60000,
        'NumOfProducts': 2,
        'HasCrCard': 1, 
        'IsActiveMember': 1,
        'EstimatedSalary': 50000
}

In [8]:
## converting the geography char data in int data using the one hot encoding
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

NameError: name 'label_encoder_geo' is not defined

In [ ]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [ ]:
## Encode categorical variables
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [ ]:
## combine one-hot encoded columns with input data
input_df = pd.concat([input_df.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [ ]:
input_scaled= scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [ ]:
## Prediction churn

prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 187ms/step


array([[0.03553427]], dtype=float32)

In [ ]:
prediction_proba = prediction[0][0]

In [ ]:
prediction_proba

0.03553427

In [ ]:
if prediction_proba > 0.5:
        print("The customer is likely to churn.")
else:
        print("The customer is not likely to churn.")

The customer is not likely to churn.
